# Finding Similar Items

## 3.1 Applications of Near-Neighbor Search

## 3.1.1 Jaccard Similarity of Sets

$$Jaccard\ Similarity = \frac{|A\cap B|}{|A\cup B|}$$

In [4]:
def jaccard_sim(a, b):
    
    return len(a.intersection(b)) / len(a.union(b))

In [5]:
a = set([1, 2, 3, 4, 5])
b = set([3, 4, 7, 8])

a_intersection_b = a.intersection(b)
a_union_b = a.union(b)

print("Intersection: {0}\nUnion: {1}\nJaccard Similarity = {2}".format(a_intersection_b, 
                                                                       a_union_b,
                                                                       jaccard_sim(a, b)))

Intersection: {3, 4}
Union: {1, 2, 3, 4, 5, 7, 8}
Jaccard Similarity = 0.2857142857142857


In [12]:
# NLTK
from nltk.metrics import *

print("Jaccard Distance: {0}\nJaccard Similarity: {1}".format(\
                                                              jaccard_distance(a, b), 
                                                              jaccard_sim(a, b)))

Jaccard distance: 0.7142857142857143
Jaccard Similarity: 0.2857142857142857


### 3.1.2 Similarity of Documents

### 3.1.3 Collaborative Filtering as a Similar-Sets Problem




||**ver introduction to recommender systems**

In [16]:
# explicar
def jaccard_bag_sim(a, b):
    
    intersection_sum = sum((a & b).values())
    union_sum = sum(a.values()) + sum(b.values())
    
    return intersection_sum / union_sum

In [17]:
from collections import Counter

a = Counter('aaab')
b = Counter('aabbc')

print("Jaccard Bag Similarity: {}".format(jaccard_bag_sim(a, b)))

Jaccard Bag Similarity: 0.3333333333333333


## 3.2 Shingling of Documents

### 3.2.1 k-Shingles

In [18]:
from nltk import ngrams

In [22]:
# character ngram
s = "Hello World"

print(list(ngrams(s, 2)))
print([''.join(i) for i in ngrams(s, 3)])

[('H', 'e'), ('e', 'l'), ('l', 'l'), ('l', 'o'), ('o', ' '), (' ', 'W'), ('W', 'o'), ('o', 'r'), ('r', 'l'), ('l', 'd')]
['Hel', 'ell', 'llo', 'lo ', 'o W', ' Wo', 'Wor', 'orl', 'rld']


In [25]:
# string ngram
s = "A disciplina Garimpagem de Dados possui 4 créditos"

print(list(ngrams(s.split(), 2)))
print([' '.join(i) for i in ngrams(s.split(), 3)])

[('A', 'disciplina'), ('disciplina', 'Garimpagem'), ('Garimpagem', 'de'), ('de', 'Dados'), ('Dados', 'possui'), ('possui', '4'), ('4', 'créditos')]
['A disciplina Garimpagem', 'disciplina Garimpagem de', 'Garimpagem de Dados', 'de Dados possui', 'Dados possui 4', 'possui 4 créditos']


In [5]:
%run books.py
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import codecs
import re
import nltk.data

In [8]:
dom_casmurro[35:37]

['O meu fim evidente era atar as duas pontas da vida, e restaurar na velhice a adolescência.',
 'Pois, senhor, não consegui recompor o que foi nem o que fui.']

In [11]:
perto_coracao[55:57]

['O homenzinho era uma pérola de bom, uma  pérola de gravata, tinha a voz grossa e dizia de dentro do bolso: "Majestade Joana, podeis me  escutardes um minuto, só um minuto podereis interromperdes vossa sempre ocupação?"',
 'E  declarava depois: "Sou vosso servo, princesa.']

In [65]:
cv = CountVectorizer(analyzer='word', # n-gram de words
                     ngram_range=(2, 2)) # 2-gram

cv.fit(dom_casmurro + perto_coracao)

print("Tamanho do vocabulário: {}".format(len(cv.vocabulary_)))

Tamanho do vocabulário: 62136


In [66]:
line = dom_casmurro[35:36]
cv_bigrams = cv.transform(line)
df_cv_bigrams = pd.DataFrame(cv_bigrams.todense(), columns=cv.get_feature_names(), index=['count']).T

print("Linha:\n{}".format(line))
df_cv_bigrams[df_cv_bigrams['count'] > 0]

Linha:
['O meu fim evidente era atar as duas pontas da vida, e restaurar na velhice a adolescência.']


,count
as duas,1
atar as,1
da vida,1
duas pontas,1
era atar,1
evidente era,1
fim evidente,1
meu fim,1
na velhice,1
pontas da,1


In [67]:
from sklearn.neighbors import KNeighborsClassifier
from scipy import sparse
import numpy as np

In [68]:
X_dom_casmurro = cv.transform(dom_casmurro)
y_dom_casmurro = np.zeros((X_dom_casmurro.shape[0], 1))
X_perto_coracao = cv.transform(perto_coracao)
y_perto_coracao = np.ones((X_perto_coracao.shape[0], 1))

X = sparse.vstack((X_dom_casmurro, X_perto_coracao))
y = np.vstack((y_dom_casmurro, y_perto_coracao)).ravel()

knn = KNeighborsClassifier().fit(X, y)

In [69]:
knn.predict_proba(cv.transform(["A menina do meio do coração"]))

array([[ 0.6,  0.4]])

In [70]:
from sklearn.metrics import accuracy_score

accuracy_score(y, knn.predict(X))

0.6652392947103275

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
tfidf = TfidfVectorizer(analyzer='word', # n-gram de words
                        stop_words = nltk.corpus.stopwords.words("portuguese")) # desconsiderar stop words

tfidf.fit(dom_casmurro + perto_coracao)

print("Tamanho do vocabulário: {}".format(len(tfidf.vocabulary_)))

Tamanho do vocabulário: 12472


In [73]:
X_dom_casmurro = tfidf.transform(dom_casmurro)
y_dom_casmurro = np.zeros((X_dom_casmurro.shape[0], 1))
X_perto_coracao = tfidf.transform(perto_coracao)
y_perto_coracao = np.ones((X_perto_coracao.shape[0], 1))

X = sparse.vstack((X_dom_casmurro, X_perto_coracao))
y = np.vstack((y_dom_casmurro, y_perto_coracao)).ravel()

knn_tfidf = KNeighborsClassifier().fit(X, y)

In [74]:
accuracy_score(y, knn_tfidf.predict(X))

0.70843828715365242

## 3.2.3 Hashing Shingles

In [75]:
from sklearn.feature_extraction.text import HashingVectorizer

In [76]:
hv = HashingVectorizer(analyzer='word', # n-gram de words
                     ngram_range=(2, 2)) # 2-gram

hv.fit([dom_casmurro, perto_coracao])

HashingVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, n_features=1048576, ngram_range=(2, 2),
         non_negative=False, norm='l2', preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None)

In [13]:
hv_bigrams = hv.transform([dom_casmurro])

In [14]:
import sys

print("Perto do Coração Selvagem vocabulary size: {0} megabytes".format(sys.getsizeof(perto_coracao) / 10**6))
print("Dom Casmurro size: {0} megabytes".format(sys.getsizeof(dom_casmurro) / 10**6))
print("CountVectorizer vocabulary size: {0} megabytes".format(sys.getsizeof(cv.vocabulary_) / 10**6))

Perto do Coração Selvagem vocabulary size: 0.582302 megabytes
Dom Casmurro size: 0.750664 megabytes
CountVectorizer vocabulary size: 2.621544 megabytes


## 3.3 Similarity-Preserving Summaries of Sets

### 3.3.1 Matrix Representation of Sets

In [15]:
from sklearn.feature_extraction import DictVectorizer

In [16]:
dv = DictVectorizer(sparse=False)

D = [{'a':1, 'd':1}, {'c': 1}, {'b': 1, 'd': 1, 'e': 1}, {'a': 1, 'c': 1, 'd': 1}]

X = dv.fit_transform(D)
X # diferente da notação no livro, os conjuntos ficam representados por linhas, e não colunas

array([[ 1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  0.]])

In [17]:
dv.inverse_transform(X)

[{'a': 1.0, 'd': 1.0},
 {'c': 1.0},
 {'b': 1.0, 'd': 1.0, 'e': 1.0},
 {'a': 1.0, 'c': 1.0, 'd': 1.0}]

### 3.3.2 Minhashing

Ver: http://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/

### 3.3.3 Minhashing and Jaccard Similarity

### 3.3.4 Minhash Signatures

### 3.3.5 Computing Minhash Signatures

## 3.4 Locality-Sensitive Hashing for Documents